## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)

In [2]:
# You can download the data in the source that is linked above the table of contents

# Read in the data
X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)
X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)

y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

## 1. Separate numerical and categorical

In [ ]:
# claim_injury_type_mapping = {
#     '4. TEMPORARY': 4,
#     '2. NON-COMP': 2,
#     '5. PPD SCH LOSS': 5,
#     '3. MED ONLY': 3,
#     '6. PPD NSL': 6,
#     '1. CANCELLED': 1,
#     '8. DEATH':8,
#     '7. PTD': 7
# }

# y_train_encoded = y_train['Claim Injury Type'].map(claim_injury_type_mapping)
# y_val_encoded = y_val['Claim Injury Type'].map(claim_injury_type_mapping)

> Separate columns in numerical and categorical

In [8]:
num_columns = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents', 'Days_between_Acc_Assembyl']

cat_columns = ['Alternative Dispute Resolution', 'Attorney/Representative', 'C-2 Date', 'C-3 Date', 'COVID-19 Indicator', 'Gender','First Hearing Date',
               'CarrierGroup_Self-insured Private Entity', 'CarrierGroup_Self-insured Public Entity', 'CarrierGroup_Special Funds',
               'CarrierGroup_State Insurance Fund', 'Industry Code_encoded_5. PPD SCH LOSS', 'Industry Code_encoded_2. NON-COMP', 
               'Industry Code_encoded_3. MED ONLY', 'Industry Code_encoded_4. TEMPORARY', 'Industry Code_encoded_1. CANCELLED', 
               'Industry Code_encoded_8. DEATH', 'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
               'Injury_Cause_Category_encoded_5. PPD SCH LOSS', 'Injury_Cause_Category_encoded_2. NON-COMP', 
               'Injury_Cause_Category_encoded_3. MED ONLY', 'Injury_Cause_Category_encoded_4. TEMPORARY', 
               'Injury_Cause_Category_encoded_1. CANCELLED', 'Injury_Cause_Category_encoded_8. DEATH','Injury_Cause_Category_encoded_6. PPD NSL',
                'Injury_Cause_Category_encoded_7. PTD', 'Nature_Injury_Occupational', 'Nature_Injury_Specific', 'Nature_Injury_Unknown',
                'Part_Body_Category_encoded_5. PPD SCH LOSS', 'Part_Body_Category_encoded_2. NON-COMP', 'Part_Body_Category_encoded_3. MED ONLY',
                'Part_Body_Category_encoded_4. TEMPORARY', 'Part_Body_Category_encoded_1. CANCELLED', 'Part_Body_Category_encoded_8. DEATH',
                'Part_Body_Category_encoded_6. PPD NSL', 'Part_Body_Category_encoded_7. PTD','Binary Average Weekly Wage']


# Create subsets
X_train_num = X_train_encoded[num_columns]
X_train_cat = X_train_encoded[cat_columns]

X_val_num = X_val_encoded[num_columns]
X_val_cat = X_val_encoded[cat_columns]

X_test_num=X_test_encoded[num_columns]
X_test_cat=X_test_encoded[cat_columns]

Convert y_train dataframe to series

In [9]:
y = y_train.squeeze()  # Converts a DataFrame with one column to a Series

## 2. Feature selection before encoding

In [13]:
import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2 as chi2_stat

# Example dataset
# X: features, y: target variable (replace these with your actual data)
# X = df.drop('target_column', axis=1)  # Your feature columns
# y = df['target_column']  # Your target column

# This is just an example, replace X and y with your actual data


# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat, y_train)

# Create a DataFrame with the results
chi2_results = pd.DataFrame({
    'Column': X_train_cat.columns,
    'Chi2': chi2_values,
    'p-value': p_values
})

# Categorize the features
important_features = chi2_results[chi2_results['p-value'] < 0.05]
not_important_features = chi2_results[chi2_results['p-value'] >= 0.05]

# Print important features

print("Important Features (p-value < 0.05):")
print(important_features)

# Print not important features
print("\nNot Important Features (p-value >= 0.05):")
print(not_important_features)


Important Features (p-value < 0.05):
                                           Column           Chi2  \
0                  Alternative Dispute Resolution    1684.142095   
1                         Attorney/Representative  102697.459068   
3                                        C-3 Date   61136.627189   
4                              COVID-19 Indicator    3731.364384   
5                                          Gender    1437.003050   
6                              First Hearing Date   86976.530245   
7        CarrierGroup_Self-insured Private Entity     836.515686   
8         CarrierGroup_Self-insured Public Entity    5846.262642   
9                      CarrierGroup_Special Funds     200.935025   
10              CarrierGroup_State Insurance Fund    4656.406713   
11          Industry Code_encoded_5. PPD SCH LOSS     676.124516   
12              Industry Code_encoded_2. NON-COMP     254.592050   
14             Industry Code_encoded_4. TEMPORARY     141.587848   
15         

In [19]:
important_features.round(5)

,Column,Chi2,p-value
0,Alternative Dispute Resolution,1684.14209,0.00000
1,Attorney/Representative,102697.45907,0.00000
3,C-3 Date,61136.62719,0.00000
4,COVID-19 Indicator,3731.36438,0.00000
5,Gender,1437.00305,0.00000
6,First Hearing Date,86976.53025,0.00000
7,CarrierGroup_Self-insured Private Entity,836.51569,0.00000
8,CarrierGroup_Self-insured Public Entity,5846.26264,0.00000
9,CarrierGroup_Special Funds,200.93502,0.00000
10,CarrierGroup_State Insurance Fund,4656.40671,0.00000


In [26]:
# Set a threshold for Chi² score
chi2_threshold = 1000

# Filter features based on Chi² score and significant p-value
selected_features = chi2_results[
    (chi2_results['Chi2'] > chi2_threshold) & 
    (chi2_results['p-value'] < 0.05)
]
selected_features

,Column,Chi2,p-value
0,Alternative Dispute Resolution,1684.142095,0.000000e+00
1,Attorney/Representative,102697.459068,0.000000e+00
3,C-3 Date,61136.627189,0.000000e+00
4,COVID-19 Indicator,3731.364384,0.000000e+00
5,Gender,1437.003050,3.799768e-306
6,First Hearing Date,86976.530245,0.000000e+00
8,CarrierGroup_Self-insured Public Entity,5846.262642,0.000000e+00
10,CarrierGroup_State Insurance Fund,4656.406713,0.000000e+00
22,Injury_Cause_Category_encoded_4. TEMPORARY,1013.468476,1.481272e-214
23,Injury_Cause_Category_encoded_1. CANCELLED,22561.513438,0.000000e+00


## 2. Data scaling

### 2.1 Normalization

In [5]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
print("Parameters fitted:")
for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
    print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
X_train_num_scaled.describe().round(2)

Parameters fitted:
Variable: Age at Injury | Min: 14.0 | Max: 83.0
Variable: Average Weekly Wage | Min: 0.0 | Max: 8.584247919248929
Variable: Birth Year | Min: 1936.0 | Max: 2008.0
Variable: IME-4 Count | Min: 0.0 | Max: 20.0
Variable: Number of Dependents | Min: 0.0 | Max: 6.0
Variable: Days_between_Acc_Assembyl | Min: 0.0 | Max: 3.4920269017768053


,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Days_between_Acc_Assembyl
count,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00
mean,0.41,0.34,0.58,0.04,0.50,0.53
std,0.20,0.40,0.19,0.09,0.33,0.17
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.25,0.00,0.43,0.00,0.17,0.42
50%,0.41,0.00,0.58,0.00,0.50,0.52
75%,0.58,0.80,0.75,0.00,0.83,0.64
max,1.00,1.00,1.00,1.00,1.00,1.00


In [6]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)
X_val_num_scaled.describe().round(2)

,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Days_between_Acc_Assembyl
count,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00
mean,0.41,0.34,0.58,0.04,0.50,0.52
std,0.20,0.40,0.19,0.10,0.33,0.17
min,0.00,0.00,-0.01,0.00,0.00,0.00
25%,0.25,0.00,0.43,0.00,0.17,0.41
50%,0.41,0.00,0.58,0.00,0.50,0.51
75%,0.58,0.80,0.74,0.00,0.83,0.63
max,1.00,1.00,1.00,1.00,1.00,0.95


In [7]:
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)
X_test_num_scaled.describe().round(2)

,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Days_between_Acc_Assembyl
count,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00
mean,0.40,0.15,0.62,0.01,0.50,0.48
std,0.20,0.32,0.19,0.04,0.33,0.16
min,0.00,0.00,-0.01,0.00,0.00,0.00
25%,0.23,0.00,0.46,0.00,0.17,0.37
50%,0.39,0.00,0.64,0.00,0.50,0.47
75%,0.57,0.00,0.79,0.00,0.83,0.58
max,1.00,1.00,1.03,0.90,1.00,0.90


### 2.2 Standarization

> We can use both, if we're using algorithms that benefit from using both.  (e.g., neural networks may benefit from normalization, while models like logistic regression may perform better with standardized data).

In [8]:
# from sklearn.preprocessing import StandardScaler

# std_scaler = StandardScaler().fit(X_train_num)
# X_train_scaled_std= scaler.transform(X_train_num)
# X_train_scaled_std = pd.DataFrame(X_train_scaled_std, columns = X_train_num.columns).set_index(X_train_encoded.index)

# X_val_scaled_std = scaler.transform(X_val_num)
# X_val_scaled_std = pd.DataFrame(X_val_scaled_std, columns = X_val_num.columns).set_index(X_val_encoded.index)


## 3. Feature Selection

### Numerical variables only

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Assuming x_train_num_scaled contains 6 numerical features
# and y_train_encoded is the target variable (encoded for classification)

# Set k to the number of available features (6 in this case)
k = 6

# Apply SelectKBest to select top k features based on ANOVA F-value
selector = SelectKBest(score_func=f_classif, k=k)  
X_new = selector.fit_transform(X_train_num_scaled, y_train_encoded)

# View the scores for each feature
print("Scores for each feature:", selector.scores_)

# If you want to see which features are selected:
selected_features = selector.get_support(indices=True)  # Get the indices of the selected features
print("Selected feature indices:", selected_features)


Scores for each feature: [8.11423330e+02 1.37542431e+05 8.76586962e+02 1.65664479e+04
 5.48779099e-01 1.06407676e+03]
Selected feature indices: [0 1 2 3 4 5]


In [22]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(X, y):
    # Create the contingency table
    df_contingency = pd.crosstab(X, y)
    
    # Perform chi-square test
    chi2, p, dof, expected = chi2_contingency(df_contingency.values)
    
    # Calculate Cramér's V
    n = df_contingency.sum().sum()  # Total number of observations
    min_dim = min(df_contingency.shape) - 1  # Min between number of rows and columns - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula
    
    return cramers_v

# Example usage:
# Assuming `X_train_cat` is your categorical feature and `y_train_encoded` is your target
for var in X_train_cat.columns:
    v = cramers_v(X_train_cat[var], y_train_encoded)
    print(f"Cramér's V for {var}: {v:.4f}")


Cramér's V for Alternative Dispute Resolution: 0.0654
Cramér's V for Attorney/Representative: 0.6149


C:\Users\emano\AppData\Local\Temp\ipykernel_11724\588541164.py:15: RuntimeWarning: invalid value encountered in divide
  cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula


Cramér's V for C-2 Date: nan
Cramér's V for C-3 Date: 0.4790
Cramér's V for COVID-19 Indicator: 0.0995
Cramér's V for Gender: 0.0938
Cramér's V for First Hearing Date: 0.5457
Cramér's V for CarrierGroup_Self-insured Private Entity: 0.0482
Cramér's V for CarrierGroup_Self-insured Public Entity: 0.1371
Cramér's V for CarrierGroup_Special Funds: 0.0225
Cramér's V for CarrierGroup_State Insurance Fund: 0.1208
Cramér's V for Industry Code_encoded_5. PPD SCH LOSS: 0.1033
Cramér's V for Industry Code_encoded_2. NON-COMP: 0.1033
Cramér's V for Industry Code_encoded_3. MED ONLY: 0.1033
Cramér's V for Industry Code_encoded_4. TEMPORARY: 0.1033
Cramér's V for Industry Code_encoded_1. CANCELLED: 0.1033
Cramér's V for Industry Code_encoded_8. DEATH: 0.1033
Cramér's V for Industry Code_encoded_6. PPD NSL: 0.1033
Cramér's V for Industry Code_encoded_7. PTD: 0.1018
Cramér's V for Injury_Cause_Category_encoded_5. PPD SCH LOSS: 0.2090
Cramér's V for Injury_Cause_Category_encoded_2. NON-COMP: 0.2090
Cram

In [23]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(X, y):
    # Create the contingency table
    df_contingency = pd.crosstab(X, y)
    
    # Perform chi-square test
    chi2, p, dof, expected = chi2_contingency(df_contingency.values)
    
    # Calculate Cramér's V
    n = df_contingency.sum().sum()  # Total number of observations
    min_dim = min(df_contingency.shape) - 1  # Min between number of rows and columns - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula
    
    return cramers_v

# Set a threshold for strong association (e.g., Cramér's V > 0.3)
threshold = 0.3
strong_features = {}

# Example usage:
# Assuming `X_train_cat` is your categorical feature matrix and `y_train_encoded` is your target
for var in X_train_cat.columns:
    v = cramers_v(X_train_cat[var], y_train_encoded)
    if v > threshold:  # Only keep features with strong association
        strong_features[var] = v

# Display the features with strong association
for feature, score in strong_features.items():
    print(f"Cramér's V for {feature}: {score:.4f}")


C:\Users\emano\AppData\Local\Temp\ipykernel_11724\2739714921.py:15: RuntimeWarning: invalid value encountered in divide
  cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula


Cramér's V for Attorney/Representative: 0.6149
Cramér's V for C-3 Date: 0.4790
Cramér's V for First Hearing Date: 0.5457
Cramér's V for Nature_Injury_Unknown: 0.4931
Cramér's V for Binary Average Weekly Wage: 0.8520


In [38]:
from sklearn.linear_model import LogisticRegression

# Fit Logistic Regression with L1 penalty
logreg = LogisticRegression(penalty='l1', solver='liblinear')
logreg.fit(X_train_cat, y_train_encoded)

# Get feature importance
feature_importance = pd.Series(logreg.coef_[0], index=X_train_cat.columns).sort_values(ascending=False)
print(feature_importance)


Injury_Cause_Category_encoded_1. CANCELLED       29.564118
Industry Code_encoded_1. CANCELLED               18.141391
Part_Body_Category_encoded_3. MED ONLY            3.226240
Attorney/Representative                           0.823494
First Hearing Date                                0.784356
Nature_Injury_Occupational                        0.343128
CarrierGroup_State Insurance Fund                 0.299789
C-2 Date                                          0.220337
Gender                                            0.076929
Industry Code_encoded_5. PPD SCH LOSS             0.052082
Part_Body_Category_encoded_8. DEATH               0.000000
Injury_Cause_Category_encoded_4. TEMPORARY        0.000000
Injury_Cause_Category_encoded_6. PPD NSL          0.000000
Injury_Cause_Category_encoded_8. DEATH            0.000000
Injury_Cause_Category_encoded_7. PTD              0.000000
Part_Body_Category_encoded_1. CANCELLED           0.000000
Part_Body_Category_encoded_5. PPD SCH LOSS        0.0000

In [ ]:
import time
from sklearn.feature_selection import mutual_info_classif

# Test on a small subset of the data first
X_train_cat_subset = X_train_cat.sample(n=100000, random_state=42)  # Just sample 1000 rows
y_train_encoded_subset = y_train_encoded.sample(n=100000, random_state=42)

start_time = time.time()
mutual_info = mutual_info_classif(X_train_cat_subset, y_train_encoded_subset)
end_time = time.time()

print(f"Time to run on 1000 rows: {end_time - start_time:.2f} seconds")

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Compute mutual information between categorical features and target
mutual_info = mutual_info_classif(X_train_cat, y_train_encoded)

# Display features sorted by mutual information
mi_results = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Mutual Information': mutual_info
}).sort_values(by='Mutual Information', ascending=False)

print(mi_results)


Time to run on 1000 rows: 58.62 seconds


In [37]:
mi_results = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Mutual Information': mutual_info
}).sort_values(by='Mutual Information', ascending=False)

print(mi_results)

                                          Feature  Mutual Information
38                     Binary Average Weekly Wage            0.444176
1                         Attorney/Representative            0.207225
6                              First Hearing Date            0.160729
3                                        C-3 Date            0.121259
21      Injury_Cause_Category_encoded_3. MED ONLY            0.075416
25       Injury_Cause_Category_encoded_6. PPD NSL            0.070568
19  Injury_Cause_Category_encoded_5. PPD SCH LOSS            0.069887
20      Injury_Cause_Category_encoded_2. NON-COMP            0.069600
22     Injury_Cause_Category_encoded_4. TEMPORARY            0.069377
23     Injury_Cause_Category_encoded_1. CANCELLED            0.069225
26           Injury_Cause_Category_encoded_7. PTD            0.069071
24         Injury_Cause_Category_encoded_8. DEATH            0.068240
33        Part_Body_Category_encoded_4. TEMPORARY            0.064201
34        Part_Body_

## SELECTED FEATURES

In [ ]:
# Remerge the numerical and categorical features
X_train = pd.concat([X_train_num_scaled[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']], 
                     X_train_cat[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]], axis=1)

X_val = pd.concat([X_val_num_scaled[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']], 
                   X_val_cat[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]], axis=1)

X_test = pd.concat([X_test_num_scaled[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']], 
                    X_test_cat[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]], axis=1)


## OVER AND UNDERSAMPLING

In [41]:
# Assuming y_train is your target variable
target_distribution = y_train.value_counts()

# Display the distribution
print(target_distribution)


Claim Injury Type
2    202245
4    101242
3     47591
5     33513
1      8191
6      2924
8       324
7        67
Name: count, dtype: int64


In [15]:
y_train

,Claim Injury Type
Claim Identifier,
5506797,5. PPD SCH LOSS
5481216,2. NON-COMP
5520188,5. PPD SCH LOSS
5748412,3. MED ONLY
5653867,4. TEMPORARY
...,...
5930802,2. NON-COMP
5757476,2. NON-COMP
5988322,2. NON-COMP


In [23]:
X_resampled_num

,IME-4 Count,Days_between_Acc_Assembyl,Age at Injury,Average Weekly Wage
0,0.000000,0.758523,0.376812,0.000000
1,0.000000,0.669530,0.565217,0.870436
2,0.000000,0.708157,0.507246,0.000000
3,0.000000,0.716502,0.188406,0.000000
4,0.000000,0.550939,0.434783,0.000000
...,...,...,...,...
328099,0.000000,0.930252,0.647640,0.904978
328100,0.192497,0.795346,0.623188,0.888554
328101,0.000000,0.599792,0.648378,0.000000
328102,0.000000,0.498854,0.509539,0.886917


In [22]:
y_resampled

0         1
1         1
2         1
3         1
4         1
         ..
328099    8
328100    8
328101    8
328102    8
328103    8
Name: Claim Injury Type, Length: 328104, dtype: int64

In [24]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Split the data into numerical and categorical features
X_train_num = X_train[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']]
X_train_cat = X_train[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]

# Define the undersampling strategy for majority classes
undersample_strategy = {
    2: 100000,   # NON-COMP: Reduce to 100k
    4: 100000,   # TEMPORARY: Reduce to 100k
    3: 47591,    # MED ONLY: Keep as is
    5: 33513,    # PPD SCH LOSS
    1: 8191,     # CANCELLED
    6: 2924,     # PPD NSL
    8: 324,      # DEATH
    7: 67        # PTD
}

# Apply undersampling to both numerical and categorical features
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)

# Apply undersampling to numerical features
X_resampled_num, y_resampled = undersampler.fit_resample(X_train_num, y_train_encoded)

# Apply undersampling to categorical features
X_resampled_cat, y_resampled = undersampler.fit_resample(X_train_cat, y_resampled)

# Check the class distribution after undersampling
print("Class distribution after undersampling:", Counter(y_resampled))

# Define SMOTE strategy for oversampling the minority classes
smote_strategy = {
    1: 30000,   # CANCELLED: Oversample to 30k
    6: 10000,   # PPD NSL: Oversample to 10k
    8: 5000,    # DEATH: Oversample to 5k
    7: 2000     # PTD: Oversample to 2k
}

# Apply SMOTE to oversample the minority classes on the numerical features
smote = SMOTE(sampling_strategy=smote_strategy, random_state=42)
X_resampled_num, y_resampled = smote.fit_resample(X_resampled_num, y_resampled)

# Apply SMOTE to oversample the minority classes on the categorical features
X_resampled_cat, y_resampled = smote.fit_resample(X_resampled_cat, y_resampled)

# Combine the resampled numerical and categorical features
X_resampled = pd.concat([X_resampled_num, X_resampled_cat], axis=1)

# Check the final class distribution after both undersampling and oversampling
print("Class distribution after SMOTE oversampling:", Counter(y_resampled))

# Optionally, you can also check the final value counts of the target labels
print(y_resampled.value_counts())


Class distribution after undersampling: Counter({2: 100000, 4: 100000, 3: 47591, 5: 33513, 1: 8191, 6: 2924, 8: 324, 7: 67})


ValueError: Found input variables with inconsistent numbers of samples: [292610, 328104]

In [ ]:
# Apply SMOTE to numerical features first
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

# Resample numerical features
X_resampled_num, y_resampled = smote.fit_resample(X_train_num, y_train_encoded)

# Apply resampling to categorical features (e.g., undersampling or other strategies)
# Assuming you're using RandomUnderSampler for categorical features
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler(random_state=42)
X_resampled_cat, _ = undersampler.fit_resample(X_train_cat, y_resampled)

# Check that the number of rows in both the resampled features and target are consistent
print(X_resampled_num.shape, X_resampled_cat.shape, y_resampled.shape)

# Now concatenate the resampled numerical and categorical features
X_resampled = pd.concat([X_resampled_num, X_resampled_cat], axis=1)


In [11]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Split into numeric and categorical features (you already have these)
X_train_num = X_train[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']]
X_train_cat = X_train[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]

# Define the oversampler and undersampler
oversample = RandomOverSampler(sampling_strategy=0.1, random_state=42)  # Oversample to 10% of majority class size
undersample = RandomUnderSampler(sampling_strategy=0.5, random_state=42)  # Majority class reduced to 50% of the total

# Oversample the numeric features
X_resampled_num, y_resampled = oversample.fit_resample(X_train_num, y_train)

# Oversample the categorical features
X_resampled_cat, y_resampled = oversample.fit_resample(X_train_cat, y_train)

# Combine resampled numeric and categorical features back into one dataset
X_resampled = pd.concat([X_resampled_num, X_resampled_cat], axis=1)

# Optionally, apply undersampling after oversampling if you need further balance
X_resampled, y_resampled = undersample.fit_resample(X_resampled, y_resampled)

# Check the class distribution after resampling
print("Class distribution after oversampling and undersampling:", Counter(y_resampled))


ValueError: "sampling_strategy" can be a float only when the type of target is binary. For multi-class, use a dict.

In [9]:
# Split into numeric and categorical features
X_train_num = X_train[['IME-4 Count', 'Days_between_Acc_Assembyl', 'Age at Injury', 'Average Weekly Wage']]
X_train_cat = X_train[['Attorney/Representative', 'C-3 Date', 'First Hearing Date', 'Nature_Injury_Unknown', 'Binary Average Weekly Wage']]

# Combine them after resampling if desired
X_resampled_num, y_resampled = oversample.fit_resample(X_train_num, y_train)  # Resample numeric features
X_resampled_cat, y_resampled = oversample.fit_resample(X_train_cat, y_train)  # Resample categorical features

# Combine back into a single dataset
X_resampled = pd.concat([X_resampled_num, X_resampled_cat], axis=1)


NameError: name 'X_train' is not defined

## 4. Model building

> Não está em 1-D o y

In [27]:
y_train = y_train_encoded
y_val = y_val_encoded

In [28]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split

# Define the models
models = {
    "MLPClassifier": MLPClassifier(max_iter=500),
    "RandomForest": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=500),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "GaussianNB": GaussianNB(),
    "RidgeClassifier": RidgeClassifier()
}

# Dictionary to store evaluation results
results = {}

# Iterate over each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Predict on the validation data
    y_pred = model.predict(X_val)
    
    # Evaluate the model
    accuracy = accuracy_score(y_val, y_pred)
    f1_macro = f1_score(y_val, y_pred, average='macro')  # Macro-averaged F1 score
    
    # Store results
    results[model_name] = {
        "accuracy": accuracy,
        "f1_macro": f1_macro
    }
    
    # Print individual model results
    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Macro F1 Score: {f1_macro:.4f}")
    print(classification_report(y_val, y_pred))  # Detailed report including precision, recall, and F1 score per class
    print("-" * 50)

# Display the summary of results
print("\nModel Evaluation Results Summary:")
for model_name, metrics in results.items():
    print(f"{model_name}: Accuracy = {metrics['accuracy']:.4f}, Macro F1 Score = {metrics['f1_macro']:.4f}")


Training MLPClassifier...
MLPClassifier Accuracy: 0.7620
MLPClassifier Macro F1 Score: 0.3318


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           1       0.63      0.31      0.42      3741
           2       0.83      0.98      0.90     87319
           3       0.47      0.05      0.08     20672
           4       0.68      0.85      0.76     44551
           5       0.59      0.42      0.49     14484
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00        29
           8       0.25      0.01      0.01       141

    accuracy                           0.76    172200
   macro avg       0.43      0.33      0.33    172200
weighted avg       0.72      0.76      0.71    172200

--------------------------------------------------
Training RandomForest...
RandomForest Accuracy: 0.7392
RandomForest Macro F1 Score: 0.3384
              precision    recall  f1-score   support

           1       0.45      0.33      0.38      3741
           2       0.84      0.95      0.89     87319
           3       0.26      0.09      0.14 

c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           1       0.56      0.26      0.36      3741
           2       0.83      0.93      0.88     87319
           3       0.35      0.05      0.09     20672
           4       0.61      0.88      0.72     44551
           5       0.49      0.20      0.29     14484
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00        29
           8       0.00      0.00      0.00       141

    accuracy                           0.73    172200
   macro avg       0.36      0.29      0.29    172200
weighted avg       0.68      0.73      0.67    172200

--------------------------------------------------
Training KNeighborsClassifier...
KNeighborsClassifier Accuracy: 0.7318
KNeighborsClassifier Macro F1 Score: 0.3284


...

c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           1       0.38      0.55      0.45      3741
           2       0.88      0.85      0.86     87319
           3       0.25      0.05      0.08     20672
           4       0.67      0.37      0.47     44551
           5       0.49      0.32      0.39     14484
           6       0.00      0.00      0.00      1263
           7       0.00      0.90      0.00        29
           8       0.01      0.84      0.02       141

    accuracy                           0.57    172200
   macro avg       0.33      0.48      0.28    172200
weighted avg       0.70      0.57      0.61    172200

--------------------------------------------------
Training RidgeClassifier...
RidgeClassifier Accuracy: 0.7263
RidgeClassifier Macro F1 Score: 0.2426


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           1       0.56      0.19      0.29      3741
           2       0.83      0.94      0.88     87319
           3       0.00      0.00      0.00     20672
           4       0.59      0.94      0.72     44551
           5       0.45      0.03      0.05     14484
           6       0.00      0.00      0.00      1263
           7       0.00      0.00      0.00        29
           8       0.00      0.00      0.00       141

    accuracy                           0.73    172200
   macro avg       0.30      0.26      0.24    172200
weighted avg       0.62      0.73      0.64    172200

--------------------------------------------------

Model Evaluation Results Summary:
MLPClassifier: Accuracy = 0.7620, Macro F1 Score = 0.3318
RandomForest: Accuracy = 0.7392, Macro F1 Score = 0.3384
LogisticRegression: Accuracy = 0.7264, Macro F1 Score = 0.2914
KNeighborsClassifier: Accuracy = 0.7318, Macro F1 Score = 0.3284
DecisionTree: Accura

c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
